In [1]:
# set HF_HOME or HF_HUB_CACHE
import os
os.environ["HF_HOME"] = "/home/234533@hertie-school.lan/workspace/cache"
os.environ["HF_HUB_CACHE"] = "/home/234533@hertie-school.lan/workspace/cache"
os.environ["TRANSFORMERS_CACHE"] = "/home/234533@hertie-school.lan/workspace/cache"

In [2]:
# Check if env variables are set correctly
print("HF_HOME:", os.environ.get("HF_HOME"))
print("HF_HUB_CACHE:", os.environ.get("HF_HUB_CACHE"))
print("TRANSFORMERS_CACHE:", os.environ.get("TRANSFORMERS_CACHE"))

HF_HOME: /home/234533@hertie-school.lan/workspace/cache
HF_HUB_CACHE: /home/234533@hertie-school.lan/workspace/cache
TRANSFORMERS_CACHE: /home/234533@hertie-school.lan/workspace/cache


In [3]:
from huggingface_hub import constants as hf_constants
from transformers.utils import TRANSFORMERS_CACHE

print("huggingface_hub cache location:", hf_constants.HF_HUB_CACHE)
print("transformers cache location:", TRANSFORMERS_CACHE)

/home/234533@hertie-school.lan/workspace/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


huggingface_hub cache location: /home/234533@hertie-school.lan/workspace/cache
transformers cache location: /home/234533@hertie-school.lan/workspace/cache


/home/234533@hertie-school.lan/workspace/venv/lib/python3.10/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
cache_dir = "/home/234533@hertie-school.lan/workspace/cache"
if os.path.exists(cache_dir):
    print(f"Cache directory exists: ✓")
    print(f"Readable: {'✓' if os.access(cache_dir, os.R_OK) else '✗'}")
    print(f"Writable: {'✓' if os.access(cache_dir, os.W_OK) else '✗'}")
else:
    print(f"Cache directory does not exist at {cache_dir}")

Cache directory exists: ✓
Readable: ✓
Writable: ✓


In [5]:
import os
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from datasets import Dataset
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
from datasets import load_dataset

In [6]:

# Load data
df = pd.read_csv('informal_reviewed.csv')
# shuffle the data
df = df.sample(frac=1).reset_index(drop=True)
texts = df['content'].tolist()
df['lable'] = df['lable'].astype(int)
labels = df['lable'].tolist()

# Create dataset function - simplified for classification
def create_dataset_entries(texts, labels=None):
    if labels:
        return [{"text": text, "label": label} for text, label in zip(texts, labels)]
    else:
        return [{"text": text} for text in texts]

# Create training dataset
data_entries = create_dataset_entries(texts, labels)
train_val_split = int(len(data_entries) * 0.9)
train_entries, val_entries = data_entries[:train_val_split], data_entries[train_val_split:]

train_dataset = Dataset.from_list(train_entries)
val_dataset = Dataset.from_list(val_entries)

print(f"Training on {len(train_dataset)} examples, validating on {len(val_dataset)} examples")

# Calculate class weights
from sklearn.utils.class_weight import compute_class_weight

# Check class distribution
class_counts = pd.Series(labels).value_counts()
print(f"Class distribution: {class_counts.to_dict()}")

# Compute balanced class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
class_weights = torch.tensor(class_weights, dtype=torch.float)
print(f"Class weights: {class_weights}")

# Define metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }



Training on 38090 examples, validating on 4233 examples
Class distribution: {0: 32302, 1: 10021}
Class weights: tensor([0.6551, 2.1117])


In [7]:

# Load model and tokenizer
model_name = "microsoft/Phi-4-mini-instruct"

# Load model from checkpoint
#checkpoint_path = "/home/234533@hertie-school.lan/workspace/phi-dogwhistle-detector/checkpoint-6000"

tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir='/home/234533@hertie-school.lan/workspace/cache')
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Process datasets with tokenizer
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Load model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    torch_dtype=torch.bfloat16,
    device_map= "auto",
    cache_dir="/home/234533@hertie-school.lan/workspace/cache",
    problem_type="single_label_classification"
)
# Configure LoRA fine-tuning
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[
        "qkv_proj",
        "o_proj",
        "gate_up_proj",
        "down_proj"]
)

# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Set up data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)




Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.71s/it]
Some weights of Phi3ForSequenceClassification were not initialized from the model checkpoint at microsoft/Phi-4-mini-instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 11,540,480 || all params: 3,847,568,384 || trainable%: 0.2999


In [8]:

# Training arguments
training_args = TrainingArguments(
    output_dir="./phi-dogwhistle-detector",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    num_train_epochs=3,
    warmup_ratio=0.05,
    fp16=False,
    bf16=True,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=600,
    save_strategy="steps",
    save_steps=600,
    save_total_limit=4,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",
    push_to_hub=False,
)

# Custom trainer for weighted loss
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        if num_items_in_batch is not None:
            loss = loss * logits.shape[0] / num_items_in_batch
        return (loss, outputs) if return_outputs else loss

# Start training from checkpoint
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# Save model
trainer.save_model("./phi-dogwhistle-detector-final")

/home/234533@hertie-school.lan/workspace/venv/lib/python3.10/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_252251/1951776949.py:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [10]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
# Prediction function
def predict_dogwhistle(text, model, tokenizer, device):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
    
    return predictions.item()

# Load the fine-tuned model for inference
tokenizer_inference = AutoTokenizer.from_pretrained("./phi-dogwhistle-detector-final")
model_inference = AutoModelForSequenceClassification.from_pretrained(
    "./phi-dogwhistle-detector-final", 
    device_map="auto"
)

# Evaluate on the entire dataset
all_predictions = []
for text in tqdm(texts, desc="Evaluating"):
    prediction = predict_dogwhistle(text, model_inference, tokenizer_inference, device)
    all_predictions.append(prediction)

# Calculate accuracy
accuracy = accuracy_score(labels, all_predictions)
print(f"Overall accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(labels, all_predictions))


Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.84s/it]
Some weights of Phi3ForSequenceClassification were not initialized from the model checkpoint at microsoft/Phi-4-mini-instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating:  16%|█▌        | 7171/45902 [12:42<1:08:39,  9.40it/s]


KeyboardInterrupt: 

In [10]:
# load device:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
# Load the external dataset
test_dataset = load_dataset("SALT-NLP/silent_signals_detection", cache_dir='/home/234533@hertie-school.lan/workspace/cache')
test_dataset = test_dataset['train']
test_df = pd.DataFrame(test_dataset)

# Preprocess the external dataset
test_raw_texts = test_df['example'].tolist()
test_labels = test_df['label'].apply(lambda x: 1 if x == 'coded' else 0).tolist()

# Load the fine-tuned model for inference
tokenizer_inference = AutoTokenizer.from_pretrained("./phi-dogwhistle-detector-final")
model_inference = AutoModelForSequenceClassification.from_pretrained(
    "./phi-dogwhistle-detector-final", 
    device_map="auto"
)

# Use the predict_dogwhistle function for external evaluation
external_predictions = []
for text in tqdm(test_raw_texts, desc="Evaluating External Dataset"):
    prediction = predict_dogwhistle(text, model_inference, tokenizer_inference, device)
    external_predictions.append(prediction)

# Evaluate predictions
accuracy = accuracy_score(test_labels, external_predictions)
print(f"External validation accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(test_labels, external_predictions))

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.37s/it]
Some weights of Phi3ForSequenceClassification were not initialized from the model checkpoint at microsoft/Phi-4-mini-instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating External Dataset: 100%|██████████| 101/101 [00:07<00:00, 13.01it/s]

External validation accuracy: 0.5149

Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.92      0.65        50
           1       0.60      0.12      0.20        51

    accuracy                           0.51       101
   macro avg       0.55      0.52      0.42       101
weighted avg       0.55      0.51      0.42       101



In [10]:
# Load RoBERTa model
import gc
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Clear GPU memory
gc.collect()
torch.cuda.empty_cache()

# Load tokenizer and model
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base", cache_dir='/home/234533@hertie-school.lan/workspace/cache')
roberta_model = RobertaForSequenceClassification.from_pretrained("roberta-base", 
                                                                 num_labels=2, 
                                                                 cache_dir='/home/234533@hertie-school.lan/workspace/cache',
                                                                 device_map="cuda:0",
                                                                 torch_dtype=torch.bfloat16)
# Tokenize dataset
def roberta_tokenize_function(examples):
    return roberta_tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

roberta_train_dataset = train_dataset.map(roberta_tokenize_function, batched=True)
roberta_val_dataset = val_dataset.map(roberta_tokenize_function, batched=True)

# Data collator
roberta_data_collator = DataCollatorWithPadding(tokenizer=roberta_tokenizer)

# Training arguments
from transformers import TrainingArguments, Trainer

roberta_training_args = TrainingArguments(
    output_dir="./roberta-dogwhistle",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=300,
    save_total_limit=2,
    learning_rate=2e-5,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# Custom trainer with class weights
class WeightedRobertaTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(logits.view(-1, model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss
    
# Instantiate trainer
roberta_trainer = WeightedRobertaTrainer(
    model=roberta_model,
    args=roberta_training_args,
    train_dataset=roberta_train_dataset,
    eval_dataset=roberta_val_dataset,
    tokenizer=roberta_tokenizer,
    data_collator=roberta_data_collator,
    compute_metrics=compute_metrics,
)

# Start training
print("Starting RoBERTa training with class weights...")
roberta_trainer.train()

# Save the trained RoBERTa model
roberta_trainer.save_model("./roberta-dogwhistle-final")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 4591/4591 [00:03<00:00, 1300.92 examples/s]
/home/234533@hertie-school.lan/workspace/venv/lib/python3.10/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_1077217/1269270439.py:57: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedRobertaTrainer.__init__`. Use `processing_class` instead.
  roberta_trainer = WeightedRobertaTrainer(


Starting RoBERTa training with class weights...


AttributeError: 'DataParallel' object has no attribute 'device'

In [11]:
# Load the fine-tuned RoBERTa model and tokenizer for inference
roberta_tokenizer_inference = RobertaTokenizer.from_pretrained("./roberta-dogwhistle-detector/checkpoint-300")
roberta_model_inference = RobertaForSequenceClassification.from_pretrained(
    "./roberta-dogwhistle-detector/checkpoint-300",
    device_map="auto"
)

def predict_dogwhistle_roberta(text, model, tokenizer, device):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=-1)
    return prediction.item()

# Evaluate RoBERTa on the full dataset
roberta_predictions = []
for text in tqdm(texts, desc="Evaluating RoBERTa"):
    pred = predict_dogwhistle_roberta(text, roberta_model_inference, roberta_tokenizer_inference, device)
    roberta_predictions.append(pred)

# Metrics
roberta_accuracy = accuracy_score(labels, roberta_predictions)
print(f"RoBERTa Accuracy: {roberta_accuracy:.4f}")
print("\nRoBERTa Classification Report:")
print(classification_report(labels, roberta_predictions))


Evaluating RoBERTa: 100%|██████████| 41717/41717 [09:50<00:00, 70.68it/s]


RoBERTa Accuracy: 0.8059

RoBERTa Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.77      0.86     31845
           1       0.55      0.92      0.69      9872

    accuracy                           0.81     41717
   macro avg       0.76      0.85      0.78     41717
weighted avg       0.87      0.81      0.82     41717

